# Collision of Two Jovian Planets

* Should I be using classes to model the particles?

## Roadmap
* Initialization
    * Initialize position
        * Use for loop, and make consecutive rings of points
        * SOMETHING ENTIRELY DIFFERENT
    * Initialize velocity
        * Uniform velocity for every particle in planet
    * Initialize mass
        * Every particle gets the same mass
            * For now? Or for always?
    * Assign density $\rho_i$
        * Use [equation 4.5] to assign density
        * $ \rho(r) = \sum_{j=1}^N \rho_j (r)$
    * Choose time scale
        * Time must satisfy the CFL condition [equation 4.12]
        
* Initaliziation (according to [equatoin 4.18])
    * Distribute particles to recover $\rho = \rho_c * sin(\zeta) / \zeta $
        
* Calculate gravitational forces $\nabla \Phi_i$
    * First pass can use brute force O(N^2) method [equation 3.1]
    * Second pass would use trees or FFT
    
* Calculate pressure $P_i$, given density $\rho_i$
    * For the outer layer, $P = \frac{R_g}{\mu} \rho T $
    * More generaly, the 'entire'?? planet can be modeled with $P = K \rho ^\gamma$ 
    * [Equations 4.14 - 4.17] go into more detail

* Calculate $\nabla P_i$
    * Can be done by combining [equations 4.3, 4.4, 4.5, and 4.9]
        * First: calculate Gaussian kernel
        * Second: multiply Gaussian kernel by mass of a specified particle j (this can be done in batch)
        * Third: Sum over local $\rho_j$ to find $\rho_i$
        * Fourth: Sum [ (mass * pressure_j/ density_j) * gradient(kernel) ]
    * Smoothing length
        * First pass: uniform
        * Second pass: locally adaptive

* Calculate change in velocity $v_i$ [equation 4.2], given pressure, density, and gravitational forces
    * This involves solving an ODE
        * First pass: RK4
        * If bad: RK1
        * And then: Leapfrog
        
* Calculate change in position $r_i$ [equation], given velocity
    * This involves solving an ODE
        * First pass: RK4
        * If bad: RK1
        * And then: Leapfrog
        
        
        
### Initialize Libraries

In [18]:
%matplotlib inline

from __future__ import division

import numpy as np
import matplotlib.pyplot as plt

### Initialize Model Constants

In [19]:
RJupiter = 6.99e9    # cm

gamma = 2
G = 6.67e-8    # dyne*cm^2/g^2

rhoC = 5    # g/cm^3, central density
PC = 6.5e13    # dyne/cm^2
TC = 22000    # K

K = 2.6e12    # dyne*cm^4/g^2

alpha = np.sqrt(K/(2*np.pi*G))

### Initialize Density

In [20]:

# for r in rspace:
#     print r/alpha

### Initialize Positions of Planets

In [21]:
N1 = 100    # Particles in Planet 1
N2 = N1     # Particles in Planet 2

# Equations 4.17
rNum = 500     
rspace = np.linspace(0, RJupiter, rNum)
zetaspace = rspace/alpha
NDistribution = []
for i in range(1,len(zetaspace)):
    zeta2 = zetaspace[i]
    zeta1 = zetaspace[i-1]
    NDistribution.append((np.sin(zeta2) - zeta2*np.cos(zeta2) - np.sin(zeta1) + zeta1*np.cos(zeta1))\
                         *N1/np.pi)
    
NDistribution = np.array(NDistribution)/NDistribution[0]  
thetadistribution = np.random.uniform(0, 2*np.pi, size=N1)

#position1, position2 = np.zeros((N1, 2)), np.zeros((N2, 2))

#rspace1 = np.linspace(0, RJupiter, N1)
#thetaspace1 = np.linspace(0, 2*np.pi, N1)

print NDistribution

[  1.00000000e+00   6.99992409e+00   1.89993927e+01   3.69976468e+01
   6.09935479e+01   9.09855778e+01   1.26971839e+02   1.68950056e+02
   2.16917571e+02   2.70871351e+02   3.30807982e+02   3.96723671e+02
   4.68614246e+02   5.46475160e+02   6.30301483e+02   7.20087912e+02
   8.15828762e+02   9.17517974e+02   1.02514911e+03   1.13871535e+03
   1.25820952e+03   1.38362403e+03   1.51495096e+03   1.65218197e+03
   1.79530838e+03   1.94432112e+03   2.09921076e+03   2.25996746e+03
   2.42658105e+03   2.59904096e+03   2.77733625e+03   2.96145564e+03
   3.15138742e+03   3.34711957e+03   3.54863966e+03   3.75593490e+03
   3.96899214e+03   4.18779786e+03   4.41233816e+03   4.64259879e+03
   4.87856511e+03   5.12022215e+03   5.36755454e+03   5.62054656e+03
   5.87918213e+03   6.14344482e+03   6.41331780e+03   6.68878391e+03
   6.96982562e+03   7.25642505e+03   7.54856395e+03   7.84622371e+03
   8.14938537e+03   8.45802962e+03   8.77213679e+03   9.09168684e+03
   9.41665940e+03   9.74703374e+03

# Notes
* Make consecutive rings to initialize position. Use a for loop to loop through radii. Increase theta every ring sampling 
    * ie: for r in rlist: for theta in range(2*r): ...